### Courno map builder

In [3]:
'''Seek for conflicts in structures eqg passed
'''
# take care about param types
# нужно получать структуру на вход по которой уже разбирать на подгруппы
#

import re
#import functions_is as fis

import numpy as np
import pandas as pd
import string # for int2base

#from itertools import product
from itertools import combinations
#from itertools import permutations
import itertools
#import timeit
#import datetime
import copy
# Dependencies (webapp)
#from ipywidgets import interact
import ipywidgets as widgets
#import io
from io import BytesIO

In [4]:
def listToString(list1):
    return (''.join(map(str, list1)))
def diversity_num(vec, dim):
    # return num of different digits without nans
    dvst = 0
    for i in range(0, dim):
        for j in range(0, len(vec)):
            if vec[j] == i:
                dvst += 1
                break
    return dvst
def list_getsome(vec, positions):
    list_n = []
    for y in positions:
        list_n.append(vec[y])
    #print("lnf", list_n)
    return(list_n)
def diversity_vec(vec, dim):
    dvst = []
    for i in range(0, dim):
        for j in range(0, len(vec)):
            if vec[j] == i:
                dvst.append(i)
                break
    return dvst
def checkinpattern_str(txt, pattern):
    # get strings
    # if work w list from begining will be faster
    ltxt = txt.split()
    stxt = set(ltxt)
    return stxt.issubset(set(pattern))

def checkinpattern_list(ltxt, lpattern):
    # get lists
    stxt = set(ltxt)
    return stxt.issubset(set(lpattern))


In [5]:
# return positions of extern () , begin from start
def openshell(f_text_v, start):
    '''return what shell contais'''
    mark = start
    Term1_r_brct_ind, Term1_l_brct_ind = 0, 0
    Term1_r_brct_cnt, Term1_l_brct_cnt = 0, 0
    while mark <= len(f_text_v): 
        if( (f_text_v[mark] == '(') and (Term1_l_brct_cnt == 0) ): 
            Term1_l_brct_ind = mark # to save pos of f l_bracket
        if(f_text_v[mark] == '('): Term1_l_brct_cnt = Term1_l_brct_cnt + 1
        if(f_text_v[mark] == ')'):
            if(Term1_r_brct_cnt >= Term1_l_brct_cnt ): return(-1) # like ...)*()
            Term1_r_brct_cnt = Term1_r_brct_cnt + 1
            Term1_r_brct_ind = mark
        if( (Term1_l_brct_cnt != 0) and (Term1_r_brct_cnt == Term1_l_brct_cnt) ): break
        mark = mark + 1
    if(Term1_l_brct_cnt != Term1_r_brct_cnt): return(-1)
    else:
        return(f_text_v[(Term1_l_brct_ind+1):Term1_r_brct_ind])


In [6]:

def structdisassemble(stct):
    los = []
    # assume braket format input, like (a(bc)). Don't see how make it simple for l10, l11 and so on
    for i in range(len(stct)):
        if stct[i] == '(': 
            t = [s for s in list(openshell(stct, i))  if s.isalpha()]
            los.append(t)
    return los

# don't think it is need to sort
def l2sort(l2):
    for i in range(0,len(l2)):
        l2[i].sort()
        


In [7]:
# test if the structure defined on set of examples can be realised 
# with incopmlete techniqe
# блоки
# 1. на основании структуры составление плана обрабоки групп
# 2. применение функции проверки/склейки к подгруппам по схеме
# адреса + функция дают разбиение на группы. На выход разбиение
# как это зациклить?
# ввожу размерность для адресов и для функции
#def eqg_og(data_ini, structure):
def eqg_groupcheck(data_ini, grouptoinvestigate):
    #функция проверки
    #составление карты Карно на основе таблицы и функции(разбиения)
    # data_ini - dataframe
    # grouptoinvestigate - names of columns to select
    #print(data_ini)
    #print(data_ini.columns)
    #alphabet = ['l1', 'l2', 'l3', 'l4', 'l5']
    alphabet = data_ini.columns
    alphabet = list(alphabet)
    alphabet.pop(len(alphabet)-1)

#     grouptoinvestigate_p = []
#     for i in range(0,len(grouptoinvestigate)):
#         grouptoinvestigate_p.append(grouptoinvestigate[i] - 1)
    #print(grouptoinvestigate)
    
    lnum = data_ini.shape[1]-1
    #print("lnum = ", lnum)
    #print(data_ini.columns[grouptoinvestigate])
        
    # w with data in pandas style
    #data = np.array(data_ini.iloc[:,grouptoinvestigate_p])
    fornames = copy.deepcopy(grouptoinvestigate)
    #fornames = grouptoinvestigate
    fornames.append('IR')
    #print(fornames)
    #print(grouptoinvestigate)
    data = data_ini[fornames]
    data_names = data.columns
    data = np.array(data_ini[grouptoinvestigate])
    #print("data \n", data)
    #reminder_slave = list(set([i for i in range(data_ini.shape[1]-1)])-set(grouptoinvestigate_p))
    reminder_slave = list(set(alphabet)-set(grouptoinvestigate))
    #print(reminder_slave)
    data_slave = np.array(data_ini[reminder_slave])
    #print("2 \n", data_slave)
    dataout = data
    # let's input numpy. And outpput too
    #data = data_ini
#     for i in range(1,3):
#         print(data[i, [i,i+1])

    # w with data in numpy style
    #dataout = data[:,grouptoinvestigate_p]
    
    # эта сортиовка нужна для вывода результатов в return.  
    dataout = np.unique(dataout, axis=0)
    
    # нужно сигнализировать если были противоречивые строки
    
    #dataout = dataout[np.lexsort((dataout[:,0], dataout[:,1], dataout[:,2], dataout[:,3]))]
    dataout = dataout[np.lexsort([dataout[:,i] for i in range(dataout.shape[1])])]
   
    
    #from operator import itemgetter
    #data1 = sorted(data1,key=itemgetter(0))
    #print("data1: \n" ,dataout)
    dim = 1+data.max()-data.min()
    dim_slave = 1+data_slave.max()-data_slave.min()
    #print('max', data.max())
    #print("dim = ", (data.max()-data.min())+1)
    #print("dim_slave = ", (data_slave.max()-data_slave.min())+1)
    dim_f = 1+data_ini['IR'].max() - data_ini['IR'].min()
    #print('dim_f', dim_f)
    #if dim_f > dim and dim_f > dim_slave:
    #    print("\n OUT OF SCALE \n")
    #print("K: " ,data[:,lnum-1])
    #print("\n")

    # all posible values of group numbers
    lenumeration = [i for i in range(0,lnum)] 

    len_combinations = 0
    for i in range(2,lnum):
        lcombinations = list(combinations(range(1, lnum+1), i))
        len_combinations += len(lcombinations)
    numofgroup = len(grouptoinvestigate)
    #print(numofgroup)
    # form table K
    #print("group of leaves investigated: ", grouptoinvestigate)#lcombinations_p[numberofvalue])
    #print("\n")
    tbl_k = np.empty((dim**numofgroup,dim_slave**(lnum-numofgroup)))
    tbl_k.fill(np.nan)
    colfilled = []
    rowfilled = []
    seen = set()
    seen_row = set()

    #enumeration_l_col = np.array(c1)
    #enumeration_l_row = np.array(c2)
    # fill table K from data string by sting (x)
    for x in range(0,len(data)):
        #print(data[x,lcombinations[j]])
        # get coordinate in nondecimal representation of string
        #print(data[x,lcombinations[numberofvalue]])
        # get coordinate in nondecimal representation of column
        
        #reminder = list(set(lenumeration)-set(grouptoinvestigate_p))#lcombinations[numberofvalue])) 
        #print(data[x,reminder])
        # need to place F value in Ktable to selected coordinates
        c1 = "".join([str(xx) for xx in data[x,:]])#lcombinations[numberofvalue]]])
        #print(c1)
        c2 = "".join([str(xx) for xx in data_slave[x,:]])
        #print(c2)
        c1_t = int(c1[::-1], base = dim)
        c2_t = int(c2[::-1], base = dim_slave)
        #print("d",c1_t)
        #print("d",c2_t)
        tbl_k[c1_t, c2_t] = data_ini.iat[x,lnum]
        # попутно можно записать все столбцы в которых есть значения, соотв. другие м.б. удал.
        if c2_t not in seen:
            seen.add(c2_t)
            colfilled.append(c2_t) 
        if c1_t not in seen_row:
            seen_row.add(c1_t)
            rowfilled.append(c1_t) 
    #print(colfilled)

    #print(rowfilled)
    #print(tbl_k)
    colfilled.sort()
    #print(colfilled)

    tbl_k_s = tbl_k[:,colfilled]
    # del nan rows
    # use rowfilled
    rows_nan = np.all(np.isnan(tbl_k_s), axis = (1))
    tbl_k_s = tbl_k_s[~rows_nan,:]
    #print(tbl_k_s)

    # отсортируй и конвертируй в нужную шкалу  colfilled
    # +1 - convert number in range from 1, for table output
    #colfilled = [i+1 for i in colfilled]
    #colfilled.sort()

    #print(colfilled)
    #rowfilled = [i+1 for i in rowfilled]
    rowfilled.sort()

    colfilled = [np.base_repr(int(i), base=dim_slave) for i in colfilled]
    colfilled.insert(0, 0)
    colfilled = ["{0:0>{1}}".format(i, data_slave.shape[1])[::-1] for i in colfilled]
    rowfilled = [np.base_repr(int(i), base=dim) for i in rowfilled]
    rowfilled = ["{0:0>{1}}".format(i, lnum-data_slave.shape[1])[::-1] for i in rowfilled]
    colfilled[0] = ''
    # print(colfilled)
    # print(rowfilled)
    temp_k = np.hstack((np.array(rowfilled).reshape(len(tbl_k_s),1), tbl_k_s))
    temp_k = np.vstack((np.array(colfilled).reshape(1,temp_k.shape[1]), temp_k))

    # create numbers of str|col 
    numbers = [int(i) for i in range(1,len(tbl_k_s)+1)]
    numbers.insert(0, "")
    numbers_col = [int(i) for i in range(0,tbl_k_s.shape[1]+1)]
    numbers_col.insert(0, "")
    numbers_col[1] = ''
    temp_k = np.hstack((np.array(numbers).reshape(len(temp_k),1), temp_k))
    temp_k = np.vstack((np.array(numbers_col).reshape(1,temp_k.shape[1]), temp_k))
    #print(tbl_k_s)
    #print(temp_k)
#     for i in range(0, temp_k.shape[0]):
#         for j in range(0, temp_k.shape[1]):
#              with data_output: print("{0:5}".format(temp_k[i][j]), end = ' ')
#     with data_output:print("\n")
    
    
    with data_output: display(pd.DataFrame(temp_k))
            
            
            
    #print(temp_k)

    # for now it is necessary to analyze whether the given alphabet is enough
    list_ind = [[i for i in range(tbl_k_s.shape[0])]]
    #print(list_ind)
    inigroup = 1

    for r in range(tbl_k_s.shape[1]):
        # to evade path dependance in table analysis - take r from end to begining

        # select a column, from back because extend used further
        # checklist - digits in curr col
        #if len(set(tbl_k_s[:,r])) > 1:
        if diversity_num(tbl_k_s[:,r], dim_f) > 1:
            # if there is conflict of type 1 = more than 1 different digit in col
            #checklist = tbl_k_s[:,r]
            checklist = copy.deepcopy(tbl_k_s[:,r])
            checklist_pos = 0
            for z in range(len(list_ind)-1,-1,-1): 
                # select a range in list_ind
                # rev list - это связано с тем что del удаляет с конца
                # в этой версии это не нужно
                ind_t = []
                flag_del_inigroup = 0
                nan_counted = 0
                for w in range(dim_f):
                    # search for every simbol in alphabet in column                      
                    range_in_range = np.where(tbl_k_s[list_ind[z],r] == w)[0].tolist()
                    #if r > 0 and len(range_in_range) and not nan_counted:
                    if not inigroup and len(range_in_range) and not nan_counted:
                        # inigroup for don't count nans when inigroup activated
                        plus_na = np.argwhere(np.isnan(tbl_k_s[list_ind[z],r]))
                        plus_na = list(itertools.chain(*plus_na))
                        range_in_range = list(itertools.chain(*[range_in_range, plus_na]))
                        nan_counted = 1
                    if len(range_in_range):
                        range_in_range = list_getsome(list_ind[z], range_in_range)
                        checklist[range_in_range] = np.nan
                        ind_t.append(range_in_range)        
                        flag_del_inigroup = 1

                if flag_del_inigroup:
                    del list_ind[z]
                    flag_del_inigroup = 0
                    inigroup = 0
                # sort list for further includes and dubles checking
                for c in range(len(ind_t)):
                    ind_t[c].sort()
                list_ind.extend(ind_t)
            # seek for opportunity to expand eqg by new digits
            # checklist_pos - digits in checklist
            if ~np.isnan(checklist).all():
                # test if there are not nans
                # каждую позицию из разнообразия нужно приклеить без противоречия
                # берем доступные числа из групп столбца и расширяем группы по ним
                checklist_pos = np.argwhere(~np.isnan(checklist))#[0].tolist()
                checklist_pos = list(itertools.chain(*checklist_pos))                
                # need to sort list_ind for groups with digits go first
                zzt = 0
                cnt_zzt = 0
                while cnt_zzt < len(list_ind)-1:
                    #if not len(set(tbl_k_s[list_ind[zzt],r])):
                    if not diversity_num(tbl_k_s[list_ind[zzt],r], dim_f):
                        #print(tbl_k_s[list_ind[zzt],r])
                        #print(len(set(tbl_k_s[list_ind[zzt],r])))
                        list_ind.append(list_ind[zzt])
                        del list_ind[zzt]
                    else: zzt += 1
                    cnt_zzt += 1
                for zz in range(len(list_ind)):
                    ind_tt = list_ind[zz]
                    cnt_j = 0 
                    for j in checklist[checklist_pos]:
                        j = int(j)
                        ldiv = diversity_vec(tbl_k_s[ind_tt,r], dim_f)
                        # проверка ГЭ на наличие символа отличного от того для которого мы ищем группу                           
                        # сформировать список без j и искать вхождение из ldiv в него
                        list_without_j = [i for i in range(dim_f)]
                        #print("dim", dim)
                        #print("j", j)
                        #print(list_without_j)
                        del list_without_j[np.nonzero(np.array(list_without_j) == j)[0][0]]
                        #print(list_without_j)
                        if any(item in ldiv for item in list_without_j):
                            cnt_j += 1
                            continue
                        else:
                            list_ind[zz].append(checklist_pos[cnt_j])
                            checklist[checklist_pos[cnt_j]] = np.nan
                            checklist_pos = np.argwhere(~np.isnan(checklist))#[0].tolist()
                            checklist_pos = list(itertools.chain(*checklist_pos))
            #else # если в ГЭ нет номера из checklist, то добваить группу
            # проверить остались ли свободные числа in checklist
            if ~np.isnan(checklist).all():
                # для каждого отдельного номинала своя группа
                ldiv2 = diversity_vec(checklist, dim_f)
                for i in ldiv2:
                    range_in_range2 = np.where(checklist == i)[0].tolist()
                    list_ind.append(range_in_range2)

    print("\n")
    list_ind_output = copy.deepcopy(list_ind)
    for i in range(len(list_ind_output)):
        for j in range(len(list_ind_output[i])):
            list_ind_output[i][j] += 1
    #print("final partition: ", list_ind_output)   
    #print("groups found = ", len(list_ind_output))
    #del list_ind_output

    #postanalisys: check if result may be merged for each pair of groups
    q = 0
    while q <= len(list_ind)-2:
        flag_onse = 1
        u = q + 1
        while u <= len(list_ind)-1:
            tbl_merged = np.vstack((tbl_k_s[list_ind[q],:], tbl_k_s[list_ind[u],:]))
            #print(tbl_merged)
            for p in range(tbl_merged.shape[1]):
                if len(set(tbl_merged[~np.isnan(tbl_merged[:,p]),p])) > 1:
                    break           
                if p == tbl_merged.shape[1]-1:
                    # was no break till ther end, so found 2 groups that can be merged
                    if flag_onse:
                        #print("in result of postanalysis found opportunity for merge")
                        flag_onse = 0
                    #print(list_ind[q], list_ind[u])
                    #t = list(itertools.chain(*[list_ind[q], list_ind[u]]))
                    list_ind.append(list(itertools.chain(*[list_ind[q], list_ind[u]])))
                    del list_ind[q]
                    del list_ind[u-1]
                    #list_ind.append(t)
                    list_ind_output = copy.deepcopy(list_ind)
                    for i in range(len(list_ind_output)):
                        for j in range(len(list_ind_output[i])):
                            list_ind_output[i][j] += 1
                    #print("final partition: ", list_ind_output)   
                    #print("groups found = ", len(list_ind_output))
                    #print("\n")
            u += 1
        q += 1
    #print(rowfilled)
    l_eqg = [0 for i in range(len(rowfilled))]
    cnt = 0
    for i in list_ind:
        for j in i:
            l_eqg[j] = cnt
        cnt += 1
    #print(l_eqg)
    #return pd.DataFrame({'A':data, 'B':l_eqg})
    #temp_k = np.hstack((np.array(rowfilled).reshape(len(tbl_k_s),1), tbl_k_s))
    dataout = np.hstack((dataout, np.array(l_eqg).reshape(len(dataout),1)))
    #data_names = list(data_names)
    #data_names = data_names.append('IS')
    #print(data_names)
    return pd.DataFrame(dataout, columns = data_names)
    #return dataout

In [ ]:
# columnsselected = widgets.Text(
#     value='ab',
#     placeholder='Type something',
#     description='Enter col splitting:',
#     disabled=False
# )
# widgets.HBox([columnsselected])

In [9]:
#out.clear_output()
data_ini = pd.read_csv('210929_id_arctic_l5_data.csv', sep=';')
print("Data preloaded: \n", data_ini.head())

Data preloaded: 
    l1  l2  l3  l4  l5  IS
0   2   1   1   2   0   2
1   1   0   0   1   0   1
2   1   1   2   0   2   2
3   2   1   2   2   1   2
4   1   1   0   1   1   2


Upload your data and chouse group for splitting, from the data column names

In [12]:
# Initialize outputs

upload_btn_output = widgets.Output(clear_output=True)
display(upload_btn_output)

split_output = widgets.Output(clear_output=True)
display(split_output)

data_process_output = widgets.Output(clear_output=True)
display(data_process_output)

compute_btn_output = widgets.Output(clear_output=True)
display(compute_btn_output)

#Print out filtered
data_output = widgets.Output(clear_output=True)
display(data_output)


#Event handler for upload button
# можно загрузить свою таблицу из файла, вызов загрузки по кнопке просто ее перезапишет

def upload_btn_eventhandler(obj):
    global data_ini
    data_process_output.clear_output(wait=True)
    for file_name in upload_file.value:
        extension = file_name.split('.')[1]
        content = upload_file.value[file_name]['content']
        if(extension == 'xlsx' or extension == 'xls'):
            data_ini = pd.read_excel(BytesIO(content))           
        elif(extension == 'csv'):
            data_ini = pd.read_csv(BytesIO(content), sep = ';')        
        else:
            print("File not accepted")
        #with data_process_output:
        #    print(f"Processed a total of {len(data_ini)} transactions.")
        with data_process_output:
            print(data_ini)
    return data_ini
                
def compute_btn_eventhandler(obj):
    global data_ini
    data_output.clear_output(wait=True)
    tablenames = list(string.ascii_lowercase[0:(data_ini.shape[1]-1)])
    tablenames.append('IR')
    #print(tablenames)
    data_ini.columns = tablenames
    #los = structdisassemble('((ab)cde)') 
    alphabet = data_ini.columns
    alphabet = list(alphabet)
    alphabet.pop(len(alphabet)-1)
    
    rmr = list(set(list(string.ascii_lowercase[0:(data_ini.shape[1]-1)]))-set(columnsselected.value))
    
    los = structdisassemble('(('+columnsselected.value+')'+str(rmr)+')') 
    
    l2sort(los)
    
    ldf = []
    ldf.append(data_ini)
    #dict_losldf
    dflosldf = pd.DataFrame(
        {
            "group": np.array(len(los), dtype="str"),
            "dfadr": np.empty(len(los), dtype="int32"),
            "crash": np.array(len(los), dtype="str"),
        }
    )
    cnt = 0
    for i in range(len(los)):
        dflosldf.iat[cnt,0] = ''.join(map(str, los[i]))
        dflosldf.iat[cnt,1] = 0
        dflosldf.iat[cnt,2] = ""
        #print("dfnumofisg[i] ", dfnumofisg.iloc[cnt,:])
        cnt += 1
    #dflosldf
    for i in range(1,len(los)):
        foundindex = i - 1 
        if i == 1:
            #print('data used: \n', ldf[i-1])
            #with data_output:
            #    display(ldf[i-1])
            #print('group used: \n', los[i])
            #with data_process_output: display('group used: \n', los[i])         
            ldf.append(eqg_groupcheck(ldf[i-1], los[i]))
            dflosldf.loc[dflosldf['group'] == ''.join(map(str, los[i])),'dfadr'] = len(ldf)-1
        else:
            # seek for dfadr to w with
            #print('['+str(''.join(map(str, los[i])))+']')
            structlikeseries = pd.Series(los[i])
            getrowslikesets = list(dflosldf['group'].apply(set))
            rowscontainstructurebool = [structlikeseries.isin(x).all() for x in getrowslikesets]
            rowscontainstructure = dflosldf['group'][rowscontainstructurebool]
            structureindexes = list(rowscontainstructure.index) # def if empty list
            structureindexes.pop() # remove last (self)
            foundindex = structureindexes.pop() # return second from end   
            #print('data used: \n', ldf[foundindex])
            #with data_output:
            #    display(ldf[foundindex])
            #with data_output:
            #    display('group used: \n', los[i])
            #print('group used: \n', los[i])
            ldf.append(eqg_groupcheck(ldf[foundindex], los[i]))
            dflosldf.loc[dflosldf['group'] == ''.join(map(str, los[i])),'dfadr'] = len(ldf)-1
        #print(dflosldf)
        # anilize F dimension and break if it > scale

        tcols = list(ldf[foundindex].columns)
        tcols.pop()

        tdf = pd.DataFrame(data=ldf[len(ldf)-1]["IR"])
        tdf.rename(columns = {'IR':'grp'}, inplace = True)
        tdf.index += 1
        #tdf = pd.DataFrame(index = pd.Index(range(1,len(ldf[len(ldf)-1]["IR"]))), data = ldf[len(ldf)-1]["IR"])
        #tdf = pd.DataFrame(data = ldf[len(ldf)-1]["IR"])
        
#         ttdf = {'index1':[i for i in range(1,len(ldf[len(ldf)-1]["IR"])+1)], "data":ldf[len(ldf)-1]["IR"]}
#         tdf = pd.DataFrame(data = ttdf)
#         tdf.set_index('index1')
        
        #tdf.set_index([pd.Index(range(1,len(tdf)+1))])
        #tdf.reindex([range(1,len(tdf)+1)])
        #tdf.reindex([i for i in range(1,len(tdf)+1)])
            #display("IR: \n", ldf[len(ldf)-1]["IR"])
        with data_output: display(tdf)
        dim_adr =  max(ldf[len(ldf)-1].iloc[0:len(ldf[len(ldf)-1])-1, 0:ldf[len(ldf)-1].shape[1]-1].max())+1       
        with data_output:
            display("dim adr =", dim_adr) #max(ldf[len(ldf)-1][0:len(ldf)-2].max())+1) 
        dim_grp = ldf[len(ldf)-1]["IR"].max()+1
        with data_output:
            display("dim grp = ", dim_grp)
        #with data_output:
            #display('---------------')
        if dim_grp > dim_adr:
            with data_output:
                display("IR out of scale")
                dflosldf.iat[i,2] = "oos"
                #display('------------------------------------------------')
    return data_ini

upload_file = widgets.FileUpload(multiple=False)
upload_btn = widgets.Button(description="Upload", icon='check', button_style='success')
upload_btn.on_click(upload_btn_eventhandler)
#data_ini = upload_btn.observe()

#out = widgets.interactive_output(upload_btn_eventhandler, {})
#display(widgets.HBox([out]))

compute_btn = widgets.Button(description="Get Courno map", icon='check', button_style='success')
compute_btn.on_click(compute_btn_eventhandler)

input_widgets = widgets.HBox([upload_file, upload_btn])
columnsselected = widgets.Text(
    value='ab',
    placeholder='Type something',
    description='column split:',
    disabled=False
)
with split_output:
    display(widgets.HBox([columnsselected]))
with compute_btn_output:
    display(compute_btn)
with upload_btn_output:
    display(input_widgets)

# download
# from ipywidgets import HTML
# from IPython.display import display

# import base64

# #res = 'computed results'
# res = data_ini
# #FILE
# filename = 'res.csv'
# b64 = base64.b64encode(res.encode())
# payload = b64.decode()

# #BUTTONS
# html_buttons = '''<html>
# <head>
# <meta name="viewport" content="width=device-width, initial-scale=1">
# </head>
# <body>
# <a download="{filename}" href="data:text/csv;base64,{payload}" download>
# <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
# </a>
# </body>
# </html>
# '''

# html_button = html_buttons.format(payload=payload,filename=filename)
# display(HTML(html_button))


Output()

Output()

Output()

Output()

Output()